# Pràctiques de Nous Usos de la Informàtica

**Nom de les persones del grup:** Pau Sanchez Valdivieso i Albert Espín Román

# Pràctica 2. Recomanadors

# Construcció d'un recomanador.

In [1]:
# lectura de dades

import pandas as pd
import numpy as np

unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::', header=None, names=rnames, engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::', header=None, names=mnames, engine='python')

data = pd.merge(pd.merge(ratings, users), movies)

# Limitem el nombre d'usuaris per a fer possible veure els resultats d'exercicis posteriors en poques hores
# Deixem en canvi totes les pel·lícules
data = data[data.user_id < 3000]

**Alerta**: Les implementacions dels exercicis 6, 7 i 8 poden tardar molt en executar-se, considera fer-ho en un subset de les dades originals. En la 1a cel·la:
```
    data = data[data.user_id < 100]
    data = data[data.movie_id < 100]
```
El codi anterior limitaria les dades a 100 usuaris i 100 películes. Recorda re-executar les cel·les.

Com a guia, una implementació que usi N usuaris i películes, per l'exercici 6, ot arribar a trigar:

* N=100, 5 segons a 60 segons
* N=1000, 15 minuts a 1 hora
* N=10000, 20 hores a 60 hores

segons la implementació utilitzada

El següent codi, donat el conjunt de dades, construeix un conjunt d'entrenament i un conjunt  de test:

In [2]:
# generem subconjunts de training i test
def assign_to_set(df):
    sampled_ids = np.random.choice(df.index,
                                   size=np.int64(np.ceil(df.index.size * 0.2)),
                                   replace=False)
    df.ix[sampled_ids, 'for_testing'] = True
    return df

data['for_testing'] = False
grouped = data.groupby('user_id', group_keys=False).apply(assign_to_set)
movielens_train = data[grouped.for_testing == False]
movielens_test = data[grouped.for_testing == True]

La següent funció `evaluate(estimate)`, donat un conjunt de dades d'entrenament i un conjunt de dades de test ens avalua la precisió d'un sistema de recomanació que li passem per paràmetre. Per a cadascun dels elements del conjunt de test haurem de pronosticar el seu valor i comparar-lo amb el valor real que l'usuari li ha asignat. La mesura que utilizarem per avaluar el sistema és la root-mean-square error (rsme)

In [3]:
# definim una funció per avaluar el resultat de la recomanació.

def compute_rmse(y_pred, y_true):
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

def evaluate(estimate,test=movielens_test):
    ids_to_estimate = zip(test['user_id'], test['movie_id'])
    estimated = np.array([estimate(u,i) for (u,i) in ids_to_estimate])
    real = test.rating.values
    nans = np.isnan(estimated)
    return compute_rmse(estimated[~nans], real[~nans])

### EXERCICI 4

+ Construeix dues funcions, `dist_euclid(x,y)` i `coef_pearson(x,y)`, que implementin la distància Euclidiana i el coeficient de correlació de Pearson entre dos vectors usant funcions de pandas. 

+ Escriu les funcions que calculin la semblança entre dos series d'un DataFrame de Pandas. S'utiltizaran per calcular les similituds entre usuaris o entre items:

    + ``def sim_euclid (data_frame, row1, row2)``
    Calcula els vectors representatius de cada fila, C1 i C2, amb les puntuacions de les columnes que estan presents en ambdós files. En el cas dels usuaris (files), això implica trobar les películes (columnes) que han puntuat tots dos.<br />Si no hi ha puntuacions en comú, retornar 0. En cas contrari, retornar ``1/(1+dist_euclid(C1, C2))``

    + ``def sim_pearson (data_frame, row1, row2)``
    Calcular els vectors representatius de cada fila, C1 i C2, amb les puntuacions de les columnes que estan presents en ambdós files.<br />Si no hi ha puntuacions en comú, retornar 0. Retornar ``coef_pearson(C1,C2)``
    

In [4]:
import math
import numpy as np
import pandas as pd
import scipy.stats as sc

# nombre mínim de pel·lícules que han d'haver valorat dos usuaris per a considerar-se
# fiable el seu valor de semblança
minimum_common_ratings = 3

def dist_euclid(x, y):
    """ Returns the euclidean distance of two vectors """
    return np.linalg.norm(np.array(x) - np.array(y))

def coef_pearson(x, y):
    """ Returns the Pearson correlation of two vectors """
    return sc.pearsonr(x, y)[0]

def clean_rows(function):
    """ Decorador que neteja les files de manera que només conserva els elements no nuls en ambdues """
    
    def wrapper(data_frame, row1, row2) :
        common = data_frame.ix[[row1, row2]].dropna(axis=1)
        
        '''fixem el requisit que dos usuaris hagin valorat un cert nombre mínim de pel·lícules
        en comú per a poder calcular un valor de similitud entre ells, altrament considerem que
        no tenim prou dades per determinar-ho rigorosament i descartem el càlcul. El valor que
        experimentalment hem trobat que funciona millor amb aquest recomanador és el d'un mínim
        de 3 pel·lícules en comú'''
        if len(common.columns) <= minimum_common_ratings:
            return np.nan
    
        return function(data_frame, common.ix[row1], common.ix[row2])
    return wrapper

@clean_rows
def sim_euclid(data_frame, row1, row2):
    """ Returns a distance-based similarity score for person1 and person2 based on euclidean distance """
    return 1 / (1 + dist_euclid(row1, row2))


@clean_rows
def sim_pearson(data_frame, row1, row2):
    """ Returns a distance-based similarity score for person1 and person2 based on pearson distance """
    return coef_pearson(row1, row2)


Tests de les funcions, pots realitzar modificacions prèvies a les taules (per exemple, agrupar o pivotar) per accelerar el procés

In [5]:
# Test de les funcions de similitud sobre els usuaris 1 i 2 (NaN indicaria que no tenen prou
# valoracions en comú com per a poder calcular un valor de similitud)

ratings = data.pivot_table(values="rating", index="user_id", columns="movie_id")

print sim_euclid(ratings, 1, 2)
print sim_pearson(ratings, 1, 2)

0.333333333333
0.416666666667


### EXERCICI 5

+ Feu dues funcions, ``get_best_euclid(data_frame, user, n)`` i ``get_best_pearson(data_frame, user, n)``, que retornin els ``n`` usuaris més semblants segons aquestes dues mesures de similitud.

In [6]:
def get_maximum_values(values, n):
    """ Obté els n majors valors presents a la llista de tuples, on el valor d'índex
        1 de la tupla és el valor numèric a tenir en compte"""
    

    '''EL següent bucle calcula els n valors màxims a la llista de tuples. En quant a la
    implementació, som conscients que es pot fer en una línia mitjançant la funció sort()
    d'ordenació i quedant-nos amb el n primers elements, però aquesta funció té complexitat
    O(m log m), sent m la mida de llista de valors. Hem preferit anar cercant el màxim de cada
    iteració i extraient-lo de la llista ja que cercar el màxim un cop és O(m), i n cops és 
    O(n * m), sent n el nombre de valors màxims a obtenir, sent generalment n un valor molt
    petit comparat amb m, podent-se aproximar a O(m), assumint que numpy implementa un algorisme
    d'eliminació d'un element prou eficient, i considerant negligibles la resta d'operacions,
    de caire senzill.
    '''
    max_values = []
    
    while len(max_values) < n and values:
        max_value = max(values, key=lambda x : x[1])
        
        if not np.isnan(max_value[1]):
            max_values.append(max_value)
        values.remove(max_value)

    return max_values


def get_best_euclid(data_frame, user, n):
    """ Retorna els n usuaris més similars a l'usuari indicat segons el criteri de similitud euclidiana """
    similarities = [(i, sim_euclid(data_frame, i, user)) for i in data_frame.index if i != user]
    return get_maximum_values(similarities, n)
       

def get_best_pearson(data_frame, user, n):
    """ Retorna els n usuaris més similars a l'usuari indicat segons el criteri de similitud de Pearson """
    similarities = [(i, sim_pearson(data_frame, i, user)) for i in data_frame.index if i != user]
    return get_maximum_values(similarities, n)


In [7]:
'''Petit test de les funcions anteriors, cercant els 5 usuaris més similars a l'usuari 1, amb
els seus valors de semblança. En cas de mostrar-se menys dels esperats es deurà a que no hi ha
suficients usuaris que hagin valorat el mateix (això només passa amb bases de dades petites o
si l'usuari ha valorat molt poques pel·lícules). Observem com el mètode de Pearson s'abstrau de
la inflació de les puntuacions, per la qual cosa resulta més versàtil.'''

print get_best_euclid(ratings, 1, 5)
print get_best_pearson(ratings, 1, 5)

[(526, 1.0), (1455, 1.0), (2426, 1.0), (2558, 1.0), (2733, 1.0)]
[(526, 1.0), (1056, 1.0), (1388, 1.0), (1455, 1.0), (2426, 1.0)]


### EXERCICI 6

En l'exercici 6 i 7 es desenvoluparà un sistema de recomanació basat en usuaris i en ítems, respectivament.

El codi donat, que es basa en 3 classes, és la recomenada per fer-ho òptim i reaprofitar el màxim de codi, però s'acceptaran solucions que no la segueixin, sempre hi quan respectin el mètode "estimate" explicat més abaix i funcionin de forma correcte.

#### `CollaborativeFiltering`

Una classe base, comuna en els 2 recomanadors, que implementarà:
  
  * `__init__`: Rep com a paràmetres el dataframe (que constarà de `user_id`, `movie_id` i `rating`), la funció de semblança (Euclidiana o Pearson) que volem usar i un paràmetre `M` que indica el tamany que tindrà la matriu de similituds.
  
  * `precompute`: Generar per cada estimació la semblança entre 2 usuaris o items seria molt costós i faria l'algorisme molt lent, per tant, aquesta funció omplirà la taula MxM (on M es el número de usuaris o items, segons el recomanador) amb el coeficient de semblança.
      * Nota: La taula es un DataFrame de Pandas, per tant accedirem als element fent servir l'indexat de Pandas (que correspon al id del user/movie, i no a la posició 0...i)
  
  * `estimate`: s'encarrega de donar la predicció, en aquest cas donat un usuari i una pel·lícula estimar el seu rating.
    + Nota 1: Si un `user_id` o `movie_id` no es troba en el DataFrame, cal retornar "np.NAN"
    + Nota 2: En el recomenador d'usuaris, s'ha d'evitar comparar `user_id` a ell mateix. De la mateixa forma, en el d'items evitarem comparar un `movie_id` amb sí mateix.

#### `UserRecomender`

Recomanador basat en usuaris que hereta de `CollaborativeFiltering`. Implementarà:

  * `__init__`: Pot realitzar transformacions al DataFrame
  
#### `ItemRecomender`

Recomanador basat en items que hereta de `CollaborativeFiltering`. Implementarà:

  * `__init__`: Pot realitzar transformacions al DataFrame
    

In [10]:
# mínim valor de semblança entre un element i un altre per tal que aquest darrer es tingui en 
# compte en el procés ponderat de recomanació (establert després d'experimentar amb diversos valors)
minimum_calculus_similarity = 0.2

class CollaborativeFiltering(object):
    """ Collaborative filtering using a custom sim(u,u'). """
    
    def __init__(self, data, M, similarity=sim_pearson):
        """ Constructor """
        self.sim_method = similarity # Gets recommendations for a person by using a weighted average
        self.ratings = data 
        self.sim = pd.DataFrame(0, index=M, columns=M)
    
    def precompute(self):
        """Prepare data structures for estimation. Compute similarity matrix self.sim"""
      
        '''Aquí preprocessem la matriu de semblances de tots els usuaris respecte tots (considerant
        similitud inclassificable, NaN, per a un usuari amb ell mateix). Com la matriu és simètrica,
        només cal fer la meitat dels càlculs, o dit d'una manera, omplir dos cel·les per càlcul.'''
    
        # Realització dels càlculs amb un diccionari
        symetry = {}
        for first_user in self.ratings.index:
            for second_user in self.ratings.index:
                if first_user != second_user:
                    if not symetry.has_key((first_user, second_user)):
                        symetry[(first_user, second_user)] = self.sim_method(self.ratings, first_user, second_user)
                        symetry[(second_user, first_user)] = symetry[(first_user, second_user)]
                else:
                    symetry[(first_user, second_user)] = np.nan
                    symetry[(second_user, first_user)] = symetry[(first_user, second_user)]
           
        # Assignació dels valors trobats a cada cel·la de la matriu de semblances
        for key, value in symetry.iteritems():
            self.sim.loc[key[0], key[1]] = value
            
    
    def estimate(self, row, col):
        """ Funció d'estimació genèrica utilitzada pels recomanadors subclasse"""
        
        # descartem els paràmetres si algun d'ells no apareix a la taula de valoracions
        if not row in self.ratings.index or not col in self.ratings.columns:
            return np.nan
        
        '''A continuació es calcula un valor de recomanació. Primerament s'escullen els elements
        canditats, deixant-se fora els massa diferentsa l'element per al qual es farà l'estimació,
        ja que experimentalment s'ha trobat més precís, i obviament descartant també un candidat
        si resulta ser l'element de l'estimació.'''
        
        row_sim_tuples = zip(self.sim.ix[row].index, self.sim.ix[row].values)
        row_sim_arrays = map(list, row_sim_tuples)
        row_sim_arrays = filter(lambda x: x[0] != row, row_sim_arrays)
        max_row_sim_arrays  = filter(lambda x : x[1] > minimum_calculus_similarity, row_sim_arrays)
        
        
        '''Un cop escollits els candidats, es calcula la suma de les seves valoracions de forma
        ponderada, és a dir, multiplicant pel seu valor de semblança amb l'element per al qual
        es fa l'estimació. Això és pot entendre (tenint en compte que la similitud màxima és 1)
        com que es dóna major credibilitat o importància als candidats considerats més similars.
        La suma de valoracions ponderades normalitzada per la suma de semblances ens dóna una
        predicció per a l'element de l'estimació, que és la que es retorna.'''
        
        for array in max_row_sim_arrays:
            
            array.append(self.ratings.loc[array[0], col])

        max_row_sim_arrays = filter(lambda x: not np.isnan(x[2]), max_row_sim_arrays)
        
        weighted_ratings_sum = 0
        sim_sum = 0
        
        for array in max_row_sim_arrays:
            weighted_ratings_sum += array[1] * array[2]
            sim_sum += array[1]
            
        recommendation_value = np.nan
        
        if sim_sum != 0:
             recommendation_value = weighted_ratings_sum / sim_sum
        
        return recommendation_value
    
    
    def get_best_recommendations(self, movie_recommendation_dict, n):
        """Es retornen les n millors pel·lícules més recomanables de les presents al diccionari"""
        
        recommendations = []
        
        '''Aquí tornem a fer valer allò que hem dit a l'exercici 5 sobre evitar un sort() a
        l'hora de calcular els n màxims per tal de reduir la complexitat (consultar l'exercici
        5 per a més detalls)'''
        while len(recommendations) < n and movie_recommendation_dict:
            movie = max(movie_recommendation_dict, key = lambda x : movie_recommendation_dict[x])
            recommendations.append(movie)
            del movie_recommendation_dict[movie]
        
        return recommendations
    
    
    def get_recommendations_with_titles(self, best_movie_ids):
        """ Retorna una tupla amb l'id i el títol de les pel·licules a recomanar. És el resultat final de l'exercici """
        return [(key, value) for key, value in dict(zip(data.movie_id.unique(), data.title.unique())).iteritems() if key in best_movie_ids]

        

In [9]:
class UserRecomender(CollaborativeFiltering):
    """ Recomender using Collaborative filtering with a User similarity (u,u'). """
    
    def __init__(self, data_train, similarity=sim_pearson):
        """ Constructor """
        
        # Matriu que conté per cada pel·lícula la puntuació de cada usuari
        transformed_data = data_train.pivot_table('rating', index='user_id', columns='movie_id')
        
        super(UserRecomender, self).__init__(transformed_data, data_train.user_id.unique(), similarity)

                
    def estimate(self, user_id, movie_id):
        """ Given an user_id and a movie_id returns the estimated rating for such movie """
        return super(UserRecomender, self).estimate(user_id, movie_id)
    
    
    def get_recomendations(self, user_id, n):
        """Funció que troba les n pel·lícules que el sistema de recomanació considera més recomanables per a l'usuari indicat"""
        all_movies = {movie_id : self.estimate(user_id, movie_id) for movie_id in self.ratings.columns}
        return self.get_best_recommendations(all_movies, n)

In [11]:
# Creació d'una instància de recomanador col·laboratiu basat en semblança d'usuaris.
# Es precomputa la matriu de semblances i es realitza una estimació d'exemple (NaN
# indicaria que no es tenen suficients dades per estimar una valoració de l'usuari
# indicat per a la pel·lícula indicada).
user_reco = UserRecomender(movielens_train)
user_reco.precompute()
user_reco.estimate(user_id=1, movie_id=1)

4.5017621392620057

In [12]:
# S'avalua l'error absolut mig comès 
mae = evaluate(user_reco.estimate, movielens_test)
print "L'error absolut mig és de", mae, "estrelles."

L'error absolut mig és de 0.96220105691 estrelles.


### EXERCICI 7

+ Desenvolupa un sistema de recomanació col·laboratiu basat en ítems. Si la classe `CollaborativeFiltering` s'ha fet prou genèrica, tan sols caldrà fer petites modificacions a `__init__`, del contrari, podeu fer les modificacions que cregueu necessàries.

In [13]:
class ItemRecomender(CollaborativeFiltering):
    """ Recomender using Collaborative filtering with a Item similarity (i,i'). """
    
    def __init__(self,data_train, similarity=sim_pearson):
        """ Constructor """
        
        # Matriu que conté per cada pel·lícula la puntuació de cada usuari; és la transposta
        # de la del recomanador basat en semblança d'usuaris
        transformed_data = data_train.pivot_table('rating', columns='user_id', index='movie_id')
        
        super(ItemRecomender, self).__init__(transformed_data, data_train.movie_id.unique(), similarity)

            
    def estimate(self, user_id, movie_id):
        """ Given an user_id and a movie_id returns the estimated rating for such movie """
        
        return super(ItemRecomender, self).estimate(movie_id, user_id)
    
    
    def get_recomendations(self, user_id, n):
        """Funció que troba les n pel·lícules que el sistema de recomanació considera més recomanables per a l'usuari indicat""" 
        all_movies = {movie_id : self.estimate(user_id, movie_id) for movie_id in self.ratings.index}
        return self.get_best_recommendations(all_movies, n)
        

In [14]:
# Creació d'una instància de recomanador col·laboratiu basat en semblança d'ítems.
# Es precomputa la matriu de semblances i es realitza una estimació d'exemple (NaN
# indicaria que no es tenen suficients dades per estimar una valoració de l'usuari
# indicat per a la pel·lícula indicada).
item_reco = ItemRecomender(movielens_train)
item_reco.precompute()
item_reco.estimate(user_id=2, movie_id=1)

3.9555374397378626

In [15]:
# S'avalua l'error absolut mig comès 
mae = evaluate(item_reco.estimate, movielens_test)
print "L'error absolut mig és de", mae, "estrelles."

L'error absolut mig és de 0.99589840811 estrelles.


### EXERCICI 8

* Feu un nou mètode `get_recomendations(user_id, n)` que retorni les n pel·lícules recomenades per a l'usuari user_id. De nou, és recomenable fer-ho a la clase pare, `CollaborativeFiltering`, cridant-la des dels fills de forma semblant a com fa `estimate`.

* Executeu la funció en els dos recomenadors 

In [16]:
recommendations = user_reco.get_recomendations(1, 5)
recommendations_with_titles = user_reco.get_recommendations_with_titles(recommendations)

# A continuació es mostren les 5 pel·lícules recomanades a l'usuari en qüestió on el primer valor és
# l'id de la pel·lícula i el segoón valor n'és el títol.
print "\nI les 5 pel·lícules recomanades són...:\n\n{}\n".format(recommendations_with_titles)



I les 5 pel·lícules recomanades són...:

[(189, 'Reckless (1995)'), (341, 'Double Happiness (1994)'), (687, 'Country Life (1994)'), (775, 'Spirits of the Dead (Tre Passi nel Delirio) (1968)'), (831, 'Stonewall (1995)')]



In [17]:
recommendations = item_reco.get_recomendations(1, 5)
recommendations_with_titles = item_reco.get_recommendations_with_titles(recommendations)

# A continuació es mostren les 5 pel·lícules recomanades a l'usuari en qüestió on el primer valor és
# l'id de la pel·lícula i el segoón valor n'és el títol.
print "\nI les 5 pel·lícules recomanades són...:\n\n{}\n".format(recommendations_with_titles)


I les 5 pel·lícules recomanades són...:

[(114, "Margaret's Museum (1995)"), (131, 'Frankie Starlight (1995)'), (183, 'Mute Witness (1994)'), (228, 'Destiny Turns on the Radio (1995)'), (336, 'Walking Dead, The (1995)')]

